# Save and Load Examples

This notebook shows how to use the load and save functionality. We're going to:
- Generate an animation
- Save that animation
- Load it to show that it works

For this example, we'll focus on using JSON. As with the rest of the notebooks, this assumes that you have a similar test setup to me.

## Generating an Animation

Since the goal is to keep this notebook as simple as possible, we're going to use the same RGB flash that we developed in another notebook:

In [ ]:
import sys

from adafruit_dotstar_pi_animation.data import *

In [ ]:
# Make 11 blue pixels
blue_pixels = [Pixel(red=0, blue=255, green=0, brightness=1.0) for i in range(11)]
blue_frame = Frame(pixels=blue_pixels, display_ms=1000)

# Make 11 red pixels
red_pixels = [Pixel(red=255, blue=0, green=0, brightness=1.0) for i in range(11)]
red_frame = Frame(pixels=red_pixels, display_ms=1000)

# Make 11 green pixels
green_pixels = [Pixel(red=0, blue=0, green=255, brightness=1.0) for i in range(11)]
green_frame = Frame(pixels=green_pixels, display_ms=1000)

animation = Animation(frames=[blue_frame, red_frame, green_frame], loop_infinitely=False, pause_between_play_ms=0, max_plays=1)

## Saving

Here, we're going to save the animation. Rather than saving it into a physical file, we're going to use an IO object to mimic a file so that we can easily inspect it.

In [ ]:
import io

from adafruit_dotstar_pi_animation.parser import JsonAnimationParser

In [ ]:
json_memory_file = io.StringIO()
JsonAnimationParser.write(json_memory_file, animation)

Let's see what's inside of our file:

In [ ]:
json_memory_file.seek(0)
json_memory_file.read()

## Reading

Great, we were able to show that we can write to JSON. Let's read a that file and show that it produces an `Animation` object and that it can be played.

In [ ]:
json_memory_file.seek(0)
animation_from_json_file = JsonAnimationParser.read(json_memory_file)

Let's see what we got:

In [ ]:
animation_from_json_file

## Playing

Let's show that animation that we read is playable:

In [ ]:
import adafruit_dotstar
import board

from adafruit_dotstar_pi_animation.player import SimplePixelBufPlayer


num_leds = 11
pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.BGR, auto_write=False)
player = SimplePixelBufPlayer(pixels)
player.load(animation_from_json_file)
player.play()

As a fun aside, let's try the binary format (backed by MessagePack):

In [ ]:
from adafruit_dotstar_pi_animation.parser import MsgPackAnimationParser

messagepack_memory_file = io.BytesIO()
MsgPackAnimationParser.write(messagepack_memory_file, animation)
messagepack_memory_file.seek(0)
animation_from_msg_pack = MsgPackAnimationParser.read(messagepack_memory_file)
animation_from_msg_pack

In [ ]:
messagepack_memory_file.seek(0)
messagepack_memory_file.read()

Let's see how big the MessagePack file is compared to the JSON file:

In [ ]:
json_memory_file.seek(0)
messagepack_memory_file.seek(0)

len(json_memory_file.read()), len(messagepack_memory_file.read())